In [1]:
import pandas as pd
import nltk
import jellyfish
import pycountry
import re

dataset = pd.read_csv('../Procesamiento_Datos/full-data.csv')

In [70]:
df_locations = pd.DataFrame(dataset.user_location.unique(), columns=['original_location'])

**Hacemos un primer barrido rápido para buscar si hay algun nombre de pais en el localización y lo guardamos en la columna 'search_country_1'**

In [72]:
exceptions = [('New\sYork', 'United States'), ('Los\sAngeles', 'United States'), 
                ('San\sFrancisco', 'United States'), ('Buenos\sAires', 'Argentina'), 
                ('Cape\sTown', 'South Africa'), ('San\sDiego', 'United States'), 
                ('Rio\sDe\sJaneiro', 'Brazil'), ('Nyc', 'United States'),('Newyork', 'United States'), ('Ny', 'United States'), 
                ('Washington\sDc', 'United States'), ('Scotland', 'United Kingdom'),
                ('Wales', 'United Kingdom'),('England', 'United Kingdom'), ('Dc', 'United States'), 
                ('Usa', 'United States'), ('Us', 'United States'), ('Uk', 'United Kingdom'), ('Queensland', 'Australia')]

In [73]:
us_states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland","Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New\sHampshire","New\sJersey","New\sMexico","New\sYork","North\sCarolina","North\sDakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode\sIsland","South\sCarolina","South\sDakota","Tennessee","Texas","Utah","Vermont","Virginia","Washington","West\sVirginia","Wisconsin","Wyoming", 
  'Al', 'Ak', 'Az', 'Ar', 'Ca', 'Co', 'Ct', 'Dc', 'De', 'Fl', 'Ga', 'Hi', 'Id', 'Il', 'In', 'Ia', 'Ks', 'Ky', 'La', 'Me', 'Md', 'Ma', 'Mi', 'Mn', 'Ms', 
  'Mo', 'Mt', 'Ne', 'Nv', 'Nh', 'Nj', 'Nm', 'Ny', 'Nc', 'Nd', 'Oh', 'Ok', 'Or', 'Pa', 'Ri', 'Sc', 'Sd', 'Tn', 'Tx', 'Ut', 'Vt', 'Va', 'Wa', 'Wv', 'Wi', 'Wy']

In [74]:
def find_country(text, exceptions):
    x = 'NA'
    text = str(text).title()
    #buscar excepciones
    for i in exceptions:
        s = re.search(i[0], text)
        if s != None:
            x = i[1]
            break

    #buscar US States
    for i in us_states:
        s = re.search(i, text)
        if s != None:
            x = 'United States'
            break

    #buscar paises
    for country in pycountry.countries:
        if country.name in text:
            x = country.name

    return x


In [75]:
df_locations['search_country_1'] = df_locations['original_location'].apply(lambda x: find_country(x, exceptions))

**A continuación, tratamos de encontrar nombres de ciudades. Para ello importamos un BBDD de ciudades del mundo**

In [76]:
list_cities = pd.read_csv('../Extracciones/otros/worldcitiespop.csv')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [77]:
list_cities.drop(['Region'], inplace=True, axis=1)


Ordenamos el dataframe de mayor a menos poplacion para que en caso de ciudades con el mismo nombre, se escoja la primera

In [78]:
list_cities.sort_values(by='Population', ascending=False, inplace=True) 
list_cities.head()

,Country,City,AccentCity,Population,Latitude,Longitude
1544449,jp,tokyo,Tokyo,31480498.0,35.685000,139.751389
570824,cn,shanghai,Shanghai,14608512.0,31.045556,121.399722
1327914,in,bombay,Bombay,12692717.0,18.975000,72.825833
2200161,pk,karachi,Karachi,11627378.0,24.905600,67.082200
1349146,in,new delhi,New Delhi,10928270.0,28.600000,77.200000


Necesitamos añadir una columna a *list_cities* con el nombre del pais. Para ello definimos la siguiente función:

In [79]:
def get_country_name(code):
    try:
        return pycountry.countries.indices['alpha_2'][str(code)].name
    except:
        return '-'

In [80]:
country_table = pd.DataFrame(list_cities.Country.unique(), columns=['code'])
country_table['Country_name'] = country_table['code'].apply(lambda x: get_country_name(x))
country_table.head()

,code,Country_name
0,jp,Japan
1,cn,China
2,in,India
3,pk,Pakistan
4,ph,Philippines


In [81]:
list_cities = pd.merge(list_cities, country_table, left_on='Country', right_on='code', how='left')
list_cities.drop('code', axis=1, inplace=True)
list_cities = list_cities[['Country', 'Country_name', 'City', 'AccentCity', 'Population', 'Latitude', 'Longitude']]

In [82]:
list_cities.head()

,Country,Country_name,City,AccentCity,Population,Latitude,Longitude
0,jp,Japan,tokyo,Tokyo,31480498.0,35.685000,139.751389
1,cn,China,shanghai,Shanghai,14608512.0,31.045556,121.399722
2,in,India,bombay,Bombay,12692717.0,18.975000,72.825833
3,pk,Pakistan,karachi,Karachi,11627378.0,24.905600,67.082200
4,in,India,new delhi,New Delhi,10928270.0,28.600000,77.200000


In [83]:
list_cities[list_cities['City']=="medina del campo"]['Country_name'].head(1).item()

'Spain'

Función para limpiar texto antes de buscar ciudad:

In [84]:
def eliminar_tildes(texto):
    texto_limpio = re.sub('(á)|(Á)|(à)|(À)','a', texto)
    texto_limpio = re.sub('(é)|(É)|(è)|(È)','e', texto_limpio)
    texto_limpio = re.sub('(í)|(Í)|(ì)|(Ì)','i', texto_limpio)
    texto_limpio = re.sub('(ó)|(Ó)|(ò)|(Ò)','o', texto_limpio)
    texto_limpio = re.sub('(ú)|(Ú)|(ù)|(Ù)','u', texto_limpio)
    return texto_limpio



In [85]:
def find_city(text):
    
    text = str(text).lower() # Paso a minusculas
    text = eliminar_tildes(text) # Eliminar tildes
    text = re.sub(r'[^\w\s]','',text) # Eliminar puntuacion
    text = text.split() # Lista con cada palabra para que no capture partes de palabras como ciudades

    try:
        for city in list_cities.City:
            if city in text:
                return city
                break
    except:
        return '-'


In [86]:
def city_to_country(city):
    try:
        return list_cities[list_cities['City']==str(city)]['Country_name'].head(1).item()
    except:
        return '-'

In [87]:
df_locations['search_city_1'] = df_locations['original_location'].apply(lambda x: find_city(x))

In [88]:
df_locations['search_city_1_country'] = df_locations['search_city_1'].apply(lambda x: city_to_country(x))

In [89]:
df_locations.tail(50)

,original_location,search_country_1,search_city_1,search_city_1_country
2641,Boxpark Wembley London,NA,london,United Kingdom
2642,Johannesburg,NA,johannesburg,South Africa
2643,Lagos Nigeria,Nigeria,lagos,Nigeria
2644,"Brighton, England",United Kingdom,brighton,United Kingdom
2645,"East, West",NA,west,Liberia
2646,All Over,United States,over,Germany
2647,Rio de Janeiro,United States,de,Burkina Faso
2648,Church,NA,church,United Kingdom
2649,he/him/they,United States,None,-
2650,Goodlife Mass,United States,None,-


In [90]:
def select_country(search_country_1, search_city_1_country):

    if search_country_1 == 'NA':
        return search_city_1_country
    else:
        return search_country_1
    
df_locations['country'] = df_locations[['search_country_1', 'search_city_1_country']].apply(lambda x: select_country(*x), axis=1)

In [91]:
def highlight_cols(s):
    return 'background-color: % s' % 'yellow'
  
df_locations.style.applymap(highlight_cols,subset = pd.IndexSlice[:, ['country']])


,original_location,search_country_1,search_city_1,search_city_1_country,country
0,"Stockholm, Sweden",Sweden,stockholm,Sweden,Sweden
1,"Namibia, Okaanga village",Namibia,village,Botswana,Namibia
2,"Leeds, England",United Kingdom,leeds,United Kingdom,United Kingdom
3,nan,NA,None,-,-
4,"Bengaluru, India",India,india,Brazil,India
5,Venice Island,NA,venice,Italy,Italy
6,Accra Ghana,Ghana,accra,Ghana,Ghana
7,"London, Ontario, Canada",Canada,london,United Kingdom,Canada
8,Roaming the Street.,NA,street,United Kingdom,United Kingdom
9,inflight,United States,None,-,United States


In [ ]:
df_locations.tail(50)